## Hard Hat Detector

### Initialize the Libraries

In [1]:
! pip list

Package                       Version
----------------------------- -----------
absl-py                       1.3.0
alembic                       1.8.1
altair                        4.2.0
anyio                         3.6.2
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
asttokens                     2.0.8
astunparse                    1.6.3
async-generator               1.10
attrs                         22.1.0
Babel                         2.10.3
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
beautifulsoup4                4.11.1
bleach                        5.0.1
blinker                       1.5
bokeh                         2.4.3
Bottleneck                    1.3.5
branca                        0.5.0
brotlipy                      0.7.0
cached-property               1.5.2
cachetools                    5.2.0
certifi                       2022.9.24
certipy                       0.1.3
cffi                          1.15.1
charset-norma

In [4]:
import cv2 as cv
from imageai.Detection import ObjectDetection

import numpy as np
import requests as req
import os as os

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

ImportError: initialization failed

### Show Window Function

In [2]:
def showImage(img):
    window_name = 'image'
    cv.imshow(window_name, img)
    cv.waitKey(0)
    cv.destroyAllWindows()

## Download Data

### Download Images of Hard Hats

In [1]:
hardhatLoc = 'http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n03492922'

hardhatImages = req.get(hardhatLoc).text
noOfImages = 0

if not os.path.exists('hardhat'):
    os.makedirs('hardhat')

for i in hardhatImages.split('\n'):
    try:
        r = req.get(i, timeout=0.5)
        file = i.split("/")[-1].split('\r')[0]
        if 'image/jpeg' in r.headers['Content-Type']:
            if len(r.content) > 8192:
                with open('hardhat\\' + file, 'wb') as outfile:
                    outfile.write(r.content)
                    noOfImages += 1
                    print('Success: ' + file)
            else:
                print('Failed: ' + file + ' -- Image too small')
        else:
            print('Failed: ' + file + ' -- Not an image')

    except Exception as e:
        print('Failed: ' + file + ' -- Error')
        
print('*********** Download Finished **************')

NameError: name 'req' is not defined

### Download Images of People

In [ ]:
peopleLoc = 'http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n07942152'

peopleImages = req.get(peopleLoc).text
noOfImages = 0

if not os.path.exists('people'):
    os.makedirs('people')

for i in peopleImages.split('\n'):
    try:
        r = req.get(i, timeout=0.5)
        file = i.split("/")[-1].split('\r')[0]
        if 'image/jpeg' in r.headers['Content-Type']:
            if len(r.content) > 8192:
                with open('people\\' + file, 'wb') as outfile:
                    outfile.write(r.content)
                    noOfImages += 1
                    print('Success: ' + file)
            else:
                print('Failed: ' + file + ' -- Image too small')
        else:
            print('Failed: ' + file + ' -- Not an image')

    except Exception as e:
        print('Failed: ' + file + ' -- Error')
        
print('*********** Download Finished **************')

### Download Pre-Train Models

In [ ]:
modelRetinaNet = 'https://github.com/OlafenwaMoses/ImageAI/releases/download/1.0/resnet50_coco_best_v2.0.1.h5'
modelYOLOv3 = 'https://github.com/OlafenwaMoses/ImageAI/releases/download/1.0/yolo.h5'
modelTinyYOLOv3 = 'https://github.com/OlafenwaMoses/ImageAI/releases/download/1.0/yolo-tiny.h5'

if not os.path.exists('yolo.h5'):
    r = req.get(modelYOLOv3, timeout=0.5)
    with open('yolo.h5', 'wb') as outfile:
        outfile.write(r.content)

## Train the Model

### Define the Detector

In [3]:
detector = ObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath('yolo.h5')
detector.loadModel()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### Clean the Data

In [4]:
hardhatImages = os.listdir("hardhat")
peopleOnly = detector.CustomObjects(person=True)

for i in hardhatImages:
    imageFile = "hardhat/{0}".format(i)
    detectedImage, detections = detector.detectCustomObjectsFromImage(custom_objects=peopleOnly, output_type="array",
                                                                      input_image=imageFile, 
                                                                      minimum_percentage_probability=30)
    if len(detections) < 0:
        os.remove(imageFile)

ValueError: Ensure you specified correct input image, input type, output type and/or output image path 

### Split the Data

In [8]:
if not os.path.exists('hardhat/train/images'):
    os.makedirs('hardhat/train/images')
if not os.path.exists('hardhat/validation/images'):
    os.makedirs('hardhat/validation/images')

hardhatImages = os.listdir("hardhat")
hardhatTrainNums = round(len(hardhatImages) * 0.90)

for i in range(0, hardhatTrainNums):
    file = "hardhat/" + hardhatImages[i]
    if os.path.isfile(file):
        os.rename(file, "hardhat/train/images/" + hardhatImages[i])
    
hardhatImages = os.listdir("hardhat")

for i in hardhatImages:
    file = "hardhat/" + i
    if os.path.isfile(file):
        os.rename(file, "hardhat/validation/images/" + i)

### Train the Model

In [21]:
from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="hardhat")

trainer.setTrainConfig(object_names_array=["person hardhat"], batch_size=4, num_experiments=200, 
                       train_from_pretrained_model="yolo.h5")

trainer.trainModel()

Generating anchor boxes for training images and annotation...
Average IOU for 9 anchors: 0.75
Anchor Boxes generated.
Detection configuration saved in  hardhat\json\detection_config.json
Training on: 	['person hardhat']
Training with Batch Size:  4
Number of Experiments:  200
Training with transfer learning from pretrained Model


E:\Development\Anaconda\envs\ImageAI\lib\site-packages\keras\callbacks\callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Epoch 1/200
944/944 [==============================] - 856s 907ms/step - loss: 36.9858 - yolo_layer_16_loss: 7.0103 - yolo_layer_17_loss: 10.9458 - yolo_layer_18_loss: 19.0297 - val_loss: 22.8127 - val_yolo_layer_16_loss: 5.6805 - val_yolo_layer_17_loss: 6.6321 - val_yolo_layer_18_loss: 12.3109

Epoch 2/200
944/944 [==============================] - 774s 820ms/step - loss: 20.2127 - yolo_layer_16_loss: 4.6473 - yolo_layer_17_loss: 5.6500 - yolo_layer_18_loss: 9.9154 - val_loss: 12.3263 - val_yolo_layer_16_loss: 5.5798 - val_yolo_layer_17_loss: 6.8109 - val_yolo_layer_18_loss: 11.0726
Epoch 3/200
944/944 [==============================] - 776s 822ms/step - loss: 18.3042 - yolo_layer_16_loss: 4.7082 - yolo_layer_17_loss: 4.9417 - yolo_layer_18_loss: 8.6543 - val_loss: 30.1679 - val_yolo_layer_16_loss: 3.5330 - val_yolo_layer_17_loss: 5.7483 - val_yolo_layer_18_loss: 10.3959
Epoch 4/200
944/944 [==============================] - 772s 817ms/step - loss: 15.8019 - yolo_layer_16_loss: 3.5019

KeyboardInterrupt: 

### Evaluate the Model

In [34]:
model05 = trainer.evaluateModel(model_path="hardhat\models\detection_model-ex-005--loss-0014.238.h5", 
                      json_path="hardhat\json\detection_config.json", iou_threshold=0.5, 
                      object_threshold=0.3, nms_threshold=0.5)
model10 = trainer.evaluateModel(model_path="hardhat\models\detection_model-ex-010--loss-0011.053.h5", 
                      json_path="hardhat\json\detection_config.json", iou_threshold=0.5, 
                      object_threshold=0.3, nms_threshold=0.5)
model15 = trainer.evaluateModel(model_path="hardhat\models\detection_model-ex-015--loss-0009.620.h5", 
                      json_path="hardhat\json\detection_config.json", iou_threshold=0.5, 
                      object_threshold=0.3, nms_threshold=0.5)
model20 = trainer.evaluateModel(model_path="hardhat\models\detection_model-ex-020--loss-0008.462.h5", 
                      json_path="hardhat\json\detection_config.json", iou_threshold=0.5, 
                      object_threshold=0.3, nms_threshold=0.5)

print('---------------------------------------------------------')
print('Iteration 05:', model05[0]['average_precision']['person hardhat'],
     'Iteration 10:', model10[0]['average_precision']['person hardhat'],
     'Iteration 15:', model15[0]['average_precision']['person hardhat'],
     'Iteration 20:', model20[0]['average_precision']['person hardhat'])
print('---------------------------------------------------------')

Starting Model evaluation....
Model File:  hardhat\models\detection_model-ex-005--loss-0014.238.h5 

Using IoU :  0.5
Using Object Threshold :  0.3
Using Non-Maximum Suppression :  0.5
person hardhat: 0.7165
mAP: 0.7165
Starting Model evaluation....
Model File:  hardhat\models\detection_model-ex-010--loss-0011.053.h5 

Using IoU :  0.5
Using Object Threshold :  0.3
Using Non-Maximum Suppression :  0.5
person hardhat: 0.7865
mAP: 0.7865
Starting Model evaluation....
Model File:  hardhat\models\detection_model-ex-015--loss-0009.620.h5 

Using IoU :  0.5
Using Object Threshold :  0.3
Using Non-Maximum Suppression :  0.5
person hardhat: 0.8355
mAP: 0.8355
Starting Model evaluation....
Model File:  hardhat\models\detection_model-ex-020--loss-0008.462.h5 

Using IoU :  0.5
Using Object Threshold :  0.3
Using Non-Maximum Suppression :  0.5
person hardhat: 0.8446
mAP: 0.8446
---------------------------------------------------------
Iteration 05: 0.7164811838244572 Iteration 10: 0.7864502281527

### Random Hard Hat Test

In [36]:
from imageai.Detection.Custom import CustomObjectDetection

detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath("hardhat\models\detection_model-ex-020--loss-0008.462.h5")
detector.setJsonPath("hardhat\json\detection_config.json")
detector.loadModel()

person hardhat  :  48.9349365234375  :  [198, 16, 253, 127]
--------------------------------
person hardhat  :  92.02308058738708  :  [84, 71, 127, 114]
--------------------------------


In [40]:
import random

testImages = os.listdir("hardhat/validation/images")
randomFile = testImages[random.randint(0, len(testImages) - 1)]

detectedImage, detections = detector.detectObjectsFromImage(output_type="array", 
                                                            input_image="hardhat/validation/images/{0}".format(randomFile), 
                                                            minimum_percentage_probability=30)
showImage(detectedImage)

for eachObject in detections:
    print(eachObject["name"] , " : ", eachObject["percentage_probability"], " : ", eachObject["box_points"] )
    print("--------------------------------")

person hardhat  :  81.50655627250671  :  [137, 74, 194, 119]
--------------------------------
person hardhat  :  64.44770097732544  :  [290, 89, 328, 123]
--------------------------------
person hardhat  :  70.66482901573181  :  [252, 90, 290, 130]
--------------------------------
person hardhat  :  54.37401533126831  :  [358, 93, 385, 131]
--------------------------------
person hardhat  :  59.80772376060486  :  [125, 100, 145, 136]
--------------------------------
person hardhat  :  45.671346783638  :  [222, 101, 249, 130]
--------------------------------
person hardhat  :  77.1479070186615  :  [314, 99, 361, 142]
--------------------------------
